<a href="https://colab.research.google.com/github/farahbakhsh3/DeepAtariLearning/blob/master/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://www.dropbox.com/s/nxz7aqtmsiusezd/atari.rar
!apt install unrar
!unrar x -r atari.rar


In [0]:
#!/usr/bin/env python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, ConvLSTM2D, MaxPool2D
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from prepare import Sample, model_def
import matplotlib.pyplot as plt


def create_model_ConvLSTM2D(INPUT_SHAPE, OUT_SHAPE, dropout_prob=0.5):
    
    model = Sequential()

    model.add(ConvLSTM2D(32, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, strides=(2, 2),
                            recurrent_dropout=dropout_prob//2, dropout=dropout_prob,
                            input_shape=INPUT_SHAPE))
#     model.add(ConvLSTM2D(32, kernel_size=(3, 3), padding='same', 
#                             activation='relu', return_sequences=True, strides=(2, 2)))
#     model.add(MaxPool2D(pool_size=(2, 2)))
#     model.add(Dropout(dropout_prob))

    model.add(ConvLSTM2D(64, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True,
                            recurrent_dropout=dropout_prob//2, dropout=dropout_prob))
    model.add(ConvLSTM2D(64, kernel_size=(3, 3), padding='same', 
                            activation='relu', return_sequences=True, strides=(2, 2),
                            recurrent_dropout=dropout_prob//2, dropout=dropout_prob))
#     model.add(MaxPool2D(pool_size=(2, 2)))
#     model.add(Dropout(dropout_prob))

#     model.add(ConvLSTM2D(128, kernel_size=(3, 3), padding='same',  
#                             activation='relu', return_sequences=True))
    model.add(ConvLSTM2D(128, kernel_size=(3, 3), padding='same', 
                            activation='relu', strides=(2, 2),
                            recurrent_dropout=dropout_prob//2, dropout=dropout_prob))
#     model.add(MaxPool2D(pool_size=(2, 2)))
#     model.add(Dropout(dropout_prob))

    model.add(Flatten())
    model.add(BatchNormalization())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(OUT_SHAPE, activation='softmax'))
    print(model.summary())

    return model


def create_model_Conv2D(INPUT_SHAPE, OUT_SHAPE, dropout_prob=0.5):

    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', 
                        activation='relu', input_shape=INPUT_SHAPE))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', 
                        activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())
    model.add(BatchNormalization())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(OUT_SHAPE, activation='softmax'))
    print(model.summary())
    
    return model

def train_generator(x_train, y_train, batch_size):
    index = _model_def.window_size
    while True:
        x = []
        y = []
        for i in range(batch_size):
            x.append(x_train[index - _model_def.window_size + i:index + i])
            y.append(y_train[index + i - 1])

        x = np.array(x)
        y = np.array(y)
        index += batch_size
        if index >= x_train.shape[0] - batch_size:
            index = _model_def.window_size

        yield x, y

def validation_generator(x_val, y_val, batch_size):
    index = _model_def.window_size
    while True:
        x = []
        y = []
        for i in range(batch_size):
            x.append(x_val[index - _model_def.window_size + i:index + i])
            y.append(y_val[index + i - 1])

        x = np.array(x)
        y = np.array(y)
        index += batch_size
        if index >= x_val.shape[0] - batch_size:
            index = _model_def.window_size

        yield x, y


if __name__ == '__main__':

    _model_def = model_def()
    x_data = np.load("X.npy")
    y_data = np.load("Y.npy")
    y_data = to_categorical(y_data, num_classes=_model_def.OUT_SHAPE)

    h = int(x_data.shape[0] * 0.9)
    x_train = x_data[:h]
    y_train = y_data[:h]
    x_val = x_data[h:]
    y_val = y_data[h:]

    epochs = 50
    batch_size = 32
    _model_def.window_size = 10

    if _model_def.model == 'Conv2D':
        model = create_model_Conv2D(_model_def.INPUT_SHAPE, _model_def.OUT_SHAPE)
    elif _model_def.model == 'ConvLSTM2D':
        model = create_model_ConvLSTM2D(_model_def.INPUT_SHAPE, _model_def.OUT_SHAPE)

    print('---------------------------')
    print('X_Train Shape: ', x_train.shape)
    print('Y_Train Shape: ', y_train.shape)
    print('---------------------------')
    print('X_Validation Shape: ', x_val.shape)
    print('Y_Validation Shape: ', y_val.shape)
    print('---------------------------')
    
    if input('Train from zero: <z>  ,  Retrain by load prev weights: <r>  ::   ') == 'r':
        model.load_weights(_model_def.weights_file)
        print('Model weights loaded : ', _model_def.weights_file) 
    else:
        print('Train from zero.')
    print('---------------------------')
    
    chekpoint = ModelCheckpoint(filepath=_model_def.weights_file, 
                                monitor='val_acc', 
                                save_best_only=True, 
                                save_weights_only=True, 
                                verbose=True)

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam', 
                    metrics=['accuracy'])
    
    if _model_def.model == 'Conv2D':
        history = model.fit(x_train, y_train, 
                            batch_size=batch_size, 
                            epochs=epochs, 
                            validation_split=0.05, 
                            callbacks=[chekpoint])

    elif _model_def.model == 'ConvLSTM2D':
        history = model.fit_generator(train_generator(x_train, y_train, batch_size), 
                            steps_per_epoch=x_train.shape[0]//batch_size,
                            epochs=epochs, 
                            validation_data=validation_generator(x_val, y_val, batch_size), 
                            validation_steps=x_val.shape[0]//batch_size,
                            callbacks=[chekpoint])

    history_dict = history.history
    acc_value = history_dict['acc']
    loss_value = history_dict['loss']
    val_acc_value = history_dict['val_acc']
    val_loss_value = history_dict['val_loss']
    epoches = range(1,len(acc_value) + 1)

    plt.plot(epoches, val_acc_value, 'r', label='Validation accuracy')
    plt.plot(epoches, acc_value, 'b', label='Train accuracy')
    plt.title('accuracy')
    plt.xlabel('Epoches')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    plt.plot(epoches, val_loss_value, 'r', label='Valdation loss')
    plt.plot(epoches, loss_value, 'b', label='Train loss')
    plt.title('loss')
    plt.xlabel('Epoches')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    print('Best weights saved. ', _model_def.weights_file)
    print('---------------------------')



In [0]:
!pip install dropbox

import dropbox
from dropbox.files import WriteMode
 
filepath = 'ConvLSTM2D_Riverraid.h5'
access_token = 'PqHDQ2OWJuYAAAAAAAAAWLZTlsb3kgWEhjrKQLeztfPalb3D9eJDueslYKgMNW7d'
dbx = dropbox.Dropbox(access_token)
f = open(filepath, 'rb')
_File = '/' + filepath
dbx.files_upload(f.read(), _File, mode=WriteMode('overwrite', None))